In [108]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
# import scipy.optimize as opt
from scipy.special import expit, logit
import scipy.stats
import pandas as pd
# import statsmodels.api as sm
# from statsmodels.stats.gof import powerdiscrepancy # modified locally, use 0.14 or see https://github.com/statsmodels/statsmodels/issues/8558
# import plotly.graph_objects as go


# Model's predictions matrices

In [109]:
def compute_t(ρ, α, β):
    """Generate t - eq .3 - the true incidence of left-handedness

    From main text:
    F_DR is the final equilibrium frequency of right-handers with allele D fixed
    We need F_DL = 1 - F_DR
    """
    if np.isclose(β, 0): 
        FDR = (1 - 2 * α + 2 * ρ) / (2 * (1 - 2 * α))  # eq 3a
    else:
        Δ = 4 * α * α - 4 * α + 4 * β * β + 1 + 8 * β * ρ
        FDR = (2 * α + 2 * β - 1 + np.sqrt(Δ)) / (4 * β)  # eq 3
        assert 0 < 1 - FDR < 1, ("t is the problem! t = ", 1-FDR)
    return 1 - FDR

In [110]:
def validate(θ):
    if len(θ) == 3:
        ρ, α, β = θ
    elif len(θ) == 2:
        ρ, α = θ
        β = 0
    else:
        raise ValueError("Length of θ is {}".format(len(θ)))

    if (not 0 <= ρ <= 1) or (not 0 <= α <= 1) or (not 0 <= β <= 1) or (ρ + α >= 0.5) or (ρ - α > 0.5) or (ρ + β > 0.5): 
        return False
    return True

In [111]:
def compute_T(ρ, α, β):
    """Generate T - column DD in Table 1

    From Appendix 3:
    T is a 3 x 2 matrix of p(Ht | Ht x Ht) entries -
    probability of a truly H child from a mating where parents are truly H
    """
    return np.array([
        # R           L
        [0.5 + ρ + α, 1 - (0.5 + ρ + α)],  # RxR
        [0.5 + ρ + β, 1 - (0.5 + ρ + β)],  # RxL
        [0.5 + ρ - α, 1 - (0.5 + ρ - α)]  # LxL
    ])

# Adjustments matrices

In [112]:
def compute_P(mp, t):

    if mp > t:

        u = (mp - t) / mp
        assert 0 <= u < 1, "u is the problem"
        
        P = np.array([
            [1, 0, 0],
            [u, 1 - u, 0],
            [u * u, 2 * u * (1 - u), (1 - u) * (1 - u)]
        ])

    if mp < t: #changed from "else"

        v = (t - mp) / (1 - mp)  # fixed from: (t - mp)/t
        # v= t/1-mp
        assert 0 <= v <= 1, "v is the problem"

        P = np.array([
            [(1 - v) * (1 - v), 2 * v * (1 - v), v * v],
            [0, 1 - v, v],
            [0, 0, 1]
        ])

    if mp == t: #added
        P = np.array([
            [1, 0, 0],
            [0, 1, 0],
            [0, 0, 1]
            ])
        assert ((0 <= P) & (P <= 1)).all(), "P is the problem"
        
    return P

In [113]:
def compute_O(mo, t):

    if mo > t:

        w = (mo - t) / (1 - t)
        assert 0 <= w < 1, "w is the problem"

        O = np.array([
            [1 - w, w],
            [0, 1]
        ])
        
    if mo < t: #changed from "else"

        x = 1 - (mo / t)
        assert 0 <= x <= 1, "x is the problem"
        
        O = np.array([
            [1, 0],
            [x, 1 - x]
        ])

    if mo == t: #added
        O = np.array([
            [1, 0],
            [0, 1]
            ])
        assert ((0 <= O) & (O <= 1)).all(), "O is the problem"

    return O

In [114]:
def compute_M(T, P, O):
    M = P @ T @ O
    assert ((0 <= M) & (M <= 1)).all(), "M is the problem"
    return M

# Data

## Observed data

In [115]:
"""Frequency of right and left handed offspring born to RxR, RxL, and LxL parental mating"""

#    R|RxR  L|RxR  R|RxL  L|RxL  R|LxL  L|LxL
#     0     1      2      3       4     5

D = [
    [841,  115,   113,   54,    1,    7],   # Ramaley 1913, paper says it does not fit
    [6917, 308,   411,   53,    18,    7],  # Chamberlain 1928
    [1842, 151,   140,   34,    5,     6],  # Rife 1940
    [140,  34,    33,    20,    8,     2],  # Merrell 1975
    [6206, 669,   471,   125,   5,     1],  # Annett 1973
    [154,  11,    31,    9,     0,     0],  # Ferronato et al. 1947
    [232,  17,    41,    7,     3,     1],  # Mascie-Taylor (unpub, in McManus 1985)
    [1060, 144,   122,   46,    3,     4],  # Chaurasia & Groswani (unpub, in McManus 1985)
    [1656, 130,   170,   40,    4,     0],  # Annett 1978
    [303,  37,    45,    15,    0,     0],  # Carter-Saltzmann 1980
    [315,  68,    57,    16,    0,     0],  # Coren & Porac 1980
    [848,  211,   325,   150,   30,   22],  # McGee & Cozad 1980
    [58,   9,     14,    5,     0,     0],  # McManus 1985 ICM1
    [134,  15,    17,    9,     1,     0],  # McManus 1985 ICM2prop
    [74,   4,     6,     2,     0,     0],  # McManus 1985 ICM2mat
    [86,   4,     8,     1,     0,     0],  # McManus 1985 ICM2pat
    [1729, 173,   281,   63,    8,     3]   # Leiber & Axelrod 1981
]
D = np.array(D)


In [116]:
"""Observed frequency of left handers in parents generation"""

mp = [
    0.0803,   # Ramaley 1913, paper says it does not fit
    0.0356,  # Chamberlain 1928
    0.0524,  # Rife 1940
    0.1553,  # Merrell 1957
    0.0440,  # Annett 1973
    0.0986,  # Ferronato et al. 1947
    0.0930,  # Mascie-Taylor (unpub, in McManus 1985)
    0.1040,  # Chaurasia & Groswani (unpub, in McManus 1985)
    0.0547,  # Annett 1978
    0.0760,  # Carter-Saltzmann 1980
    0.0838,  # Coren & Porac 1980
    0.1802,  # McGee & Cozad 1980
    0.1105,  # McManus 1985 ICM1
    0.0795,  # McManus 1985 ICM2prop
    0.0465,  # McManus 1985 ICM2mat
    0.0455,  # McManus 1985 ICM2pat
    0.0811 # Leiber & Axelrod 1981
]

mp = np.array(mp)


In [117]:
"""Observed frequency of left handers in offspring generation"""
mo = [
    0.1556,   # Ramaley 1913, paper says it does not fit
    0.0477,  # Chamberlain 1928
    0.0877,  # Rife 1940
    0.2362,  # Merrell 1957
    0.1063,  # Annett 1973
    0.0976,  # Ferronato et al. 1947
    0.0831,  # Mascie-Taylor (unpub, in McManus 1985)
    0.1407,  # Chaurasia & Groswani (unpub, in McManus 1985)
    0.0850,  # Annett 1978
    0.1320,  # Carter-Saltzmann 1980
    0.1838,  # Coren & Porac 1980
    0.2457,  # McGee & Cozad 1980
    0.1628,  # McManus 1985 ICM1
    0.1364,  # McManus 1985 ICM2prop
    0.0698,  # McManus 1985 ICM2mat
    0.0505,  # McManus 1985 ICM2pat
    0.1059 # Leiber & Axelrod 1981
]

mo=np.array(mo)


In [118]:
Lalalnd_index = ['Ramaley 1913','Chamberlain 1928','Rife 1940','Merrell 1957', 'Annett 1973', 'Ferronato et al. 1947', 
                                 'Mascie-Taylor (unpub)', 'Chaurasia & Goswani (unpub)', 'Annett 1978', 'Carter-Saltzmann 1980', 'Coren & Porac 1980',
                                  'McGee & Cozad 1980', 'McManus 1985 ICM1', 'McManus 1985 ICM2prop', 'McManus 1985 ICM2mat', 'McManus 1985 ICM2pat',
                                'Leiber & Axelrod 1981']

## Laland (1995) predictions

In [119]:
""" Laland's expected values"""

#    R|RxR  L|RxR     R|RxL    L|RxL   R|LxL    L|LxL
#     0       1        2         3       4        5
    
Laland_95_prediction = [
    [826.7,  129.3,   123.3,   43.7,    4.9,     3.1],   # Ramaley 1913, paper says it does not fit
    [6906.8, 318.3,   419.8,   44.2,    21.3,    3.7],  # Chamberlain 1928
    [1838.3, 154.7,   143.8,   30.2,    8.03,    3.0],  # Rife 1940
    [137.8,  36.2,    37.2,    15.8,    6.1,     3.9],  # Merrell 1975
    [6214.0, 661.0,   469.9,   126.1,   4.0,     2.0],  # Annett 1973
    [152.6,  12.5,    32.5,    7.5,     0.0,     0.0],  # Ferronato et al. 1947
    [232.7,  16.3,    40.3,    7.7,     3.0,     1.0],  # Mascie-Taylor (unpub, in McManus 1985)
    [1067.7, 136.3,   126.8,   41.3,    4.4,     2.7],  # Chaurasia & Groswani (unpub, in McManus 1985)
    [1652.4, 133.6,   174.7,   35.3,    3.0,     1.0],  # Annett 1978
    [301.8,  38.2,    45.5,    14.5,    0.0,     0.0],  # Carter-Saltzmann 1980
    [320.5,  62.5,    52.1,    20.9,    0.0,     0.0],  # Coren & Porac 1980
    [828.1,  230.9,   334.9,   140.1,   32.7,   19.3],  # McGee & Cozad 1980
    [58.3,   8.7,     14.0,    5.0,     0.0,     0.0],  # McManus 1985 ICM1
    [131.7,  17.3,    19.6,    6.4,     0.6,     0.4],  # McManus 1985 ICM2prop
    [73.1,   4.9,     6.9,     1.1,     0.0,     0.0],  # McManus 1985 ICM2mat
    [85.9,   4.1,     8.1,     0.9,     0.0,     0.0],  # McManus 1985 ICM2pat
    [1737.5, 164.5,   273.1,   70.9,    7.4,     3.6]   # Leiber & Axelrod 1981
]

Laland_95_prediction = np.array(Laland_95_prediction)

In [176]:
"""Laland (1995) G-statistic values"""
G_1995_each = np.array([13.19,5.09, 4.37, 3.33, 0.90,0.54, 0.11, 2.27, 3.26, 0.06, 2.25, 3.80, 0.01, 2.61, 0.88, 0.01, 1.75])
G_1995_overall = G_1995_each.sum().round(2)
G_1995_overall_wrong =  44.33

## Parameter valus

In [190]:

#            rho    alpha   beta
thetas = [
            [0.267, 0.148,  0.012], # Laland_95_3d  0
            [0.277, 0.138,  0],     # Laland_95_2d  1
            [0.267, 0.148,  0.012], # MLE_T_3d      2
            [0.277, 0.138,  0],     # MLE_T_2d      3
            [0.207, 0.203,  0.042], # MLE M 3D (with Mcmanus mp,mo) 4
            [0.239, 0.172,  0]      # MLE M 2D (with Mcmanus mp,mo) 5
            #   [0.205, 0.206,  0.040], # MLE_M_3d
            #   [0.235, 0.177,  0]      ## MLE_M_2d

            ]

thetas = np.array(thetas)

# G-test for goodness-of-fit

In [122]:
""" Generate expected table for the data using the adjustments in appendix 3:"""

def expected_table(θ, Observed, adjust):

    exp_table = np.array([]) 

    if len(θ) == 3:
        ρ, α, β = θ
    elif len(θ) == 2:
        ρ, α = θ
        β = 0
    
    t=compute_t(*θ)
    T=compute_T(*θ)
    
    if adjust == True:

        for index, dataset in enumerate(Observed):
            mp_idx = mp[index]
            mo_idx = mo[index]
            assert 0 < mp_idx < 1, "mp is not right"
            assert 0 < mo_idx < 1, "mo is not right"

            P = compute_P(mp_idx, t)
            assert ((0 <= P) & (P <= 1)).all()
        
            O = compute_O(mo_idx, t)
            assert ((0 <= O) & (O <= 1)).all()
        
            M = compute_M(T, P, O)
            assert ((0 <= M) & (M <= 1)).all()

            Obs= dataset.copy()
            Exp= M.reshape(6).copy()

            Exp[:2] *= Obs[:2].sum()
            Exp[2:4] *= Obs[2:4].sum()
            Exp[4:] *= Obs[4:].sum()
            Exp[Exp==0] = 1e-16

            exp_table = np.append(exp_table, Exp)
    
    if adjust == False:
            
        for dataset in Observed:
            Obs= dataset.copy()
            Exp= T.reshape(6).copy()

            Exp[:2] *= Obs[:2].sum()
            Exp[2:4] *= Obs[2:4].sum()
            Exp[4:] *= Obs[4:].sum()
            Exp[Exp==0] = 1e-16

            exp_table = np.append(exp_table, Exp)

    return exp_table


In [165]:
def G_test(Observed, Expected, df):

    Data_sum= Observed.sum()
    Expected_sum= round(Expected.sum(),0)
    assert (Data_sum == Expected_sum), ("observed and expected sums are not equal")

    G = 0
    
    for i in range(0, len(Observed)):
        if Observed[i] > 0: # else: G += 0
            G += Observed[i] * np.log(Observed[i] / Expected[i])


    G = 2*G 
    p_value = scipy.stats.chi2.sf(G, df)

    G = G.round(2)

    if p_value < 0.05:
        print("G= {:.3}*,\tp_value= {:.3f}".format(G, p_value))
    else:
        print("G= {:.3},\tp_value= {:.3f}".format(G, p_value))

    return G , p_value

In [155]:
# Goodness of fit test for overall dataset and for each data (row)

def Goodness_of_fit(θ, Observed, adjust):
    
    """ calculating degrees of freedom:
    Each study in the data is a 2x3 table of (child | parent X parent), therefore df per row is (2-1)(3-1) = 2 and for the entire data 17*2 = 34.
    As θ is estimated across the entire data, the number of parameters in θ should be substracted so that df_overall = 34 - (number of parameters in θ).
    For each study,  df_each=   (34 - number of parameters in θ)/17 """

    if (len(θ) == 3 and θ[2] == 0) or (len(θ) == 2):    # if beta was fixed to zero (θ includ 2 parameters: rho, alpha)
        df_overall = 32    # df_overall = 34-2
        df_each = 1.88     # df_each = (34-2)/17
    else:                                               # if beta not fixed to zero (θ includ 3 parameters: rho, alpha, beta)
        df_overall = 31    # df_overall = 34-2 
        df_each = 1.82  #df_each = (34-3)/17

    print('G-test for ρ={:.3f}, α={:.3f}, β={:.3f}'.format(*θ))
    print('df overall = ',df_overall, ' df each = ', df_each , '\n')
    print("-"*30)


# Overall
    print("Overall:\n")
    Exp_all = expected_table(θ, Observed, adjust) 
    Obs_all = Observed.reshape(Exp_all.shape)
    G_all, p_all = G_test(Obs_all,  Exp_all, df = df_overall)
    Delta_overall = (G_1995_overall - G_all.round(2)).round(3)

    print("-"*30)

# Each study
    print("Each row in dataset:\n")
    Exp_each = Exp_all.reshape(Observed.shape)
    Obs_each = Observed.copy()
    G_each = np.array([])
    p_each = np.array([])
    Delta_each = np.array([])

    for i in range(0,len(Observed)):
        G_idx, pvalue_idx = G_test(Obs_each[i], Exp_each[i],df = df_each)
        G_each = np.append(G_each,G_idx)
        p_each = np.append(p_each,pvalue_idx)
        Delta_each = np.append (Delta_each, (G_1995_each[i] - G_idx.round(2))).round(3)
    
    print("-"*30)
    
    return  G_all, p_all, Delta_overall, G_each, p_each, Delta_each

In [125]:
""" Table of differences between prediction and observed data """

def results_table(θ, Observed, adjust, delta):
    
    prediction = expected_table(θ, Observed, adjust) #generate a table of our predictions shped (102,)
    prediction = prediction.round(1)
    observed = Observed.reshape(prediction.shape) #reshape observed table into (102,)

    if delta == True:
        """ Return a table where each cell is: observed-prediction """
        Deltas = np.array([]) 
        for index, observed_value in enumerate(observed):
            diff = (observed_value - prediction[index])
            Deltas = np.append(arr = Deltas, values= diff)
        Deltas = Deltas.reshape(17,6)

        df = pd.DataFrame(data = Deltas,
                          index = Lalalnd_index,
                          columns=pd.MultiIndex.from_product([['RxR','RxL','LxL',], ['R', 'L']]))
        df['G'] = G_each.round(2)

    if delta == False:
        """ Return a table where each cell is: observed value (predicted value) """ 
        forplot = np.array([])
        for index, observed_value in enumerate(observed):
            observed_idx = '{}'.format(observed_value)
            predicted_idx = '({:.1f})'.format(prediction[index])
             
            cell = observed_idx+' '+predicted_idx
            forplot = np.append(forplot, cell)
        
        forplot = forplot.reshape(17,6)

        df = pd.DataFrame(data = forplot,
                          index = Lalalnd_index,
                          columns=pd.MultiIndex.from_product([['RxR','RxL','LxL',], ['R', 'L']]))
        df['G'] = G_each.round(2)

    return df

# Results

* To see the difference between Laland's prediction anf ours set 'Observed' in the function 'result table' to Laland prediction table

## Step 1: G-test without adjustments for MLE using the matrix T

### 3D

In [204]:
""" for estimated of  ρ=0.267, α=0.148, β=0.012 """

G_all, p_all, Delta_overall, G_each, p_each, Delta_each = Goodness_of_fit(thetas[0,:], D, adjust= False)

print('Delta overall=', Delta_overall)
print('Delta each dataset=', Delta_each)
print("Biggest diff =", abs(Delta_each).max())
print("Smallest diff =", abs(Delta_each).min())

res_step1_3D = results_table(thetas[0,:], D, adjust= False, delta= False)
res_step1_3D['G_Laland'] = G_1995_each
res_step1_3D['diff_G'] = Delta_each
res_step1_3D

# res_step1_3D.to_excel('/Users/ronykarstadt/Desktop/Laland et at/results of G test feb 2024/results G-test step 1_3D.xlsx')

G-test for ρ=0.267, α=0.148, β=0.012
df overall =  31  df each =  1.82 

------------------------------
Overall:

G= 5.56e+02*,	p_value= 0.000
------------------------------
Each row in dataset:

G= 31.4*,	p_value= 0.000
G= 2.38e+02*,	p_value= 0.000
G= 4.16,	p_value= 0.107
G= 28.7*,	p_value= 0.000
G= 14.6*,	p_value= 0.001
G= 0.77,	p_value= 0.636
G= 3.0,	p_value= 0.195
G= 20.2*,	p_value= 0.000
G= 8.6*,	p_value= 0.011
G= 2.58,	p_value= 0.242
G= 33.0*,	p_value= 0.000
G= 1.56e+02*,	p_value= 0.000
G= 2.0,	p_value= 0.329
G= 3.53,	p_value= 0.148
G= 1.35,	p_value= 0.464
G= 3.01,	p_value= 0.194
G= 4.42,	p_value= 0.094
------------------------------
Delta overall= -511.3
Delta each dataset= [-1.8230e+01 -2.3278e+02  2.1000e-01 -2.5410e+01 -1.3710e+01 -2.3000e-01
 -2.8900e+00 -1.7980e+01 -5.3400e+00 -2.5200e+00 -3.0750e+01 -1.5263e+02
 -1.9900e+00 -9.2000e-01 -4.7000e-01 -3.0000e+00 -2.6700e+00]
Biggest diff = 232.78
Smallest diff = 0.21


RxR                       RxL  \
                                         R            L            R   
Ramaley 1913                   841 (874.7)   115 (81.3)  113 (130.1)   
Chamberlain 1928             6917 (6610.9)  308 (614.1)  411 (361.5)   
Rife 1940                    1842 (1823.6)  151 (169.4)  140 (135.5)   
Merrell 1957                   140 (159.2)    34 (14.8)    33 (41.3)   
Annett 1973                  6206 (6290.6)  669 (584.4)  471 (464.3)   
Ferronato et al. 1947          154 (151.0)    11 (14.0)    31 (31.2)   
Mascie-Taylor (unpub)          232 (227.8)    17 (21.2)    41 (37.4)   
Chaurasia & Goswani (unpub)  1060 (1101.7)  144 (102.3)  122 (130.9)   
Annett 1978                  1656 (1634.2)  130 (151.8)  170 (163.6)   
Carter-Saltzmann 1980          303 (311.1)    37 (28.9)    45 (46.7)   
Coren & Porac 1980             315 (350.4)    68 (32.6)    57 (56.9)   
McGee & Cozad 1980             848 (969.0)   211 (90.0)  325 (370.0)   
McManus 1985 ICM1                58 (61.3)      9 (5.7)    14 (14.8)   
McManus 1985 ICM2prop          134 (136.3)    15 (12.7)    17 (20.3)   
McManus 1985 ICM2mat             74 (71.4)      4 (6.6)      6 (6.2)   
McManus 1985 ICM2pat             86 (82.4)      4 (7.6)      8 (7.0)   
Leiber & Axelrod 1981        1729 (1740.3)  173 (161.7)  281 (268.0)   

                                                LxL                  G  \
                                       L          R          L           
Ramaley 1913                   54 (36.9)    1 (5.0)    7 (3.0)   31.42   
Chamberlain 1928              53 (102.5)  18 (15.5)    7 (9.5)  237.87   
Rife 1940                      34 (38.5)    5 (6.8)    6 (4.2)    4.16   
Merrell 1957                   20 (11.7)    8 (6.2)    2 (3.8)   28.74   
Annett 1973                  125 (131.7)    5 (3.7)    1 (2.3)   14.61   
Ferronato et al. 1947            9 (8.8)    0 (0.0)    0 (0.0)    0.77   
Mascie-Taylor (unpub)           7 (10.6)    3 (2.5)    1 (1.5)    3.00   
Chaurasia & Goswani (unpub)    46 (37.1)    3 (4.3)    4 (2.7)   20.25   
Annett 1978                    40 (46.4)    4 (2.5)    0 (1.5)    8.60   
Carter-Saltzmann 1980          15 (13.3)    0 (0.0)    0 (0.0)    2.58   
Coren & Porac 1980             16 (16.1)    0 (0.0)    0 (0.0)   33.00   
McGee & Cozad 1980           150 (105.0)  30 (32.2)  22 (19.8)  156.43   
McManus 1985 ICM1                5 (4.2)    0 (0.0)    0 (0.0)    2.00   
McManus 1985 ICM2prop            9 (5.7)    1 (0.6)    0 (0.4)    3.53   
McManus 1985 ICM2mat             2 (1.8)    0 (0.0)    0 (0.0)    1.35   
McManus 1985 ICM2pat             1 (2.0)    0 (0.0)    0 (0.0)    3.01   
Leiber & Axelrod 1981          63 (76.0)    8 (6.8)    3 (4.2)    4.42   

                            G_Laland  diff_G  
                                              
Ramaley 1913                   13.19  -18.23  
Chamberlain 1928                5.09 -232.78  
Rife 1940                       4.37    0.21  
Merrell 1957                    3.33  -25.41  
Annett 1973                     0.90  -13.71  
Ferronato et al. 1947           0.54   -0.23  
Mascie-Taylor (unpub)           0.11   -2.89  
Chaurasia & Goswani (unpub)     2.27  -17.98  
Annett 1978                     3.26   -5.34  
Carter-Saltzmann 1980           0.06   -2.52  
Coren & Porac 1980              2.25  -30.75  
McGee & Cozad 1980              3.80 -152.63  
McManus 1985 ICM1               0.01   -1.99  
McManus 1985 ICM2prop           2.61   -0.92  
McManus 1985 ICM2mat            0.88   -0.47  
McManus 1985 ICM2pat            0.01   -3.00  
Leiber & Axelrod 1981           1.75   -2.67

In [203]:
17-8

9

### 2D

In [205]:
""" for estimated of  ρ=0.277, α=0.128, β=0 """

G_all, p_all, Delta_overall, G_each, p_each, Delta_each = Goodness_of_fit(thetas[1,:], D, adjust= False)

print('Delta overall=', Delta_overall)
print('Delta each dataset=', Delta_each)
print("Biggest diff =", abs(Delta_each).max())
print("Smallest diff =", abs(Delta_each).min())

res_step1_2D = results_table(thetas[1,:], Laland_95_prediction, adjust= False, delta= False)
res_step1_2D['G_Laland'] = G_1995_each
res_step1_2D['diff_G'] = Delta_each
res_step1_2D

# res_step1_2D.to_excel('/Users/ronykarstadt/Desktop/Laland et at/results of G test feb 2024/results G-test step 1_2D.xlsx')

G-test for ρ=0.277, α=0.138, β=0.000
df overall =  32  df each =  1.88 

------------------------------
Overall:

G= 5.56e+02*,	p_value= 0.000
------------------------------
Each row in dataset:

G= 31.7*,	p_value= 0.000
G= 2.39e+02*,	p_value= 0.000
G= 4.6,	p_value= 0.090
G= 28.3*,	p_value= 0.000
G= 14.6*,	p_value= 0.001
G= 0.77,	p_value= 0.653
G= 3.0,	p_value= 0.204
G= 20.3*,	p_value= 0.000
G= 8.5*,	p_value= 0.012
G= 2.54,	p_value= 0.258
G= 33.0*,	p_value= 0.000
G= 1.56e+02*,	p_value= 0.000
G= 1.98,	p_value= 0.346
G= 3.39,	p_value= 0.167
G= 1.35,	p_value= 0.481
G= 3.03,	p_value= 0.201
G= 4.54,	p_value= 0.093
------------------------------
Delta overall= -511.6
Delta each dataset= [-1.8530e+01 -2.3355e+02 -2.3000e-01 -2.4930e+01 -1.3670e+01 -2.3000e-01
 -2.8900e+00 -1.8020e+01 -5.2400e+00 -2.4800e+00 -3.0750e+01 -1.5206e+02
 -1.9700e+00 -7.8000e-01 -4.7000e-01 -3.0200e+00 -2.7900e+00]
Biggest diff = 233.55
Smallest diff = 0.23


RxR                           RxL  \
                                           R              L              R   
Ramaley 1913                   826.7 (874.7)   129.3 (81.3)  123.3 (129.8)   
Chamberlain 1928             6906.8 (6611.0)  318.3 (614.1)  419.8 (360.5)   
Rife 1940                    1838.3 (1823.6)  154.7 (169.4)  143.8 (135.2)   
Merrell 1957                   137.8 (159.2)    36.2 (14.8)    37.2 (41.2)   
Annett 1973                  6214.0 (6290.6)  661.0 (584.4)  469.9 (463.1)   
Ferronato et al. 1947          152.6 (151.1)    12.5 (14.0)    32.5 (31.1)   
Mascie-Taylor (unpub)          232.7 (227.8)    16.3 (21.2)    40.3 (37.3)   
Chaurasia & Goswani (unpub)  1067.7 (1101.7)  136.3 (102.3)  126.8 (130.6)   
Annett 1978                  1652.4 (1634.2)  133.6 (151.8)  174.7 (163.2)   
Carter-Saltzmann 1980          301.8 (311.1)    38.2 (28.9)    45.5 (46.6)   
Coren & Porac 1980             320.5 (350.4)    62.5 (32.6)    52.1 (56.7)   
McGee & Cozad 1980             828.1 (969.0)   230.9 (90.0)  334.9 (369.1)   
McManus 1985 ICM1                58.3 (61.3)      8.7 (5.7)    14.0 (14.8)   
McManus 1985 ICM2prop          131.7 (136.3)    17.3 (12.7)    19.6 (20.2)   
McManus 1985 ICM2mat             73.1 (71.4)      4.9 (6.6)      6.9 (6.2)   
McManus 1985 ICM2pat             85.9 (82.4)      4.1 (7.6)      8.1 (7.0)   
Leiber & Axelrod 1981        1737.5 (1740.3)  164.5 (161.7)  273.1 (267.3)   

                                                    LxL                    G  \
                                         L            R            L           
Ramaley 1913                   43.7 (37.2)    4.9 (5.1)    3.1 (2.9)   31.72   
Chamberlain 1928              44.2 (103.5)  21.3 (16.0)    3.7 (9.0)  238.64   
Rife 1940                      30.2 (38.8)   8.03 (7.0)    3.0 (4.0)    4.60   
Merrell 1957                   15.8 (11.8)    6.1 (6.4)    3.9 (3.6)   28.26   
Annett 1973                  126.1 (132.9)    4.0 (3.8)    2.0 (2.2)   14.57   
Ferronato et al. 1947            7.5 (8.9)    0.0 (0.0)    0.0 (0.0)    0.77   
Mascie-Taylor (unpub)           7.7 (10.7)    3.0 (2.6)    1.0 (1.4)    3.00   
Chaurasia & Goswani (unpub)    41.3 (37.5)    4.4 (4.5)    2.7 (2.6)   20.29   
Annett 1978                    35.3 (46.8)    3.0 (2.6)    1.0 (1.4)    8.50   
Carter-Saltzmann 1980          14.5 (13.4)    0.0 (0.0)    0.0 (0.0)    2.54   
Coren & Porac 1980             20.9 (16.3)    0.0 (0.0)    0.0 (0.0)   33.00   
McGee & Cozad 1980           140.1 (105.9)  32.7 (33.2)  19.3 (18.8)  155.86   
McManus 1985 ICM1                5.0 (4.2)    0.0 (0.0)    0.0 (0.0)    1.98   
McManus 1985 ICM2prop            6.4 (5.8)    0.6 (0.6)    0.4 (0.4)    3.39   
McManus 1985 ICM2mat             1.1 (1.8)    0.0 (0.0)    0.0 (0.0)    1.35   
McManus 1985 ICM2pat             0.9 (2.0)    0.0 (0.0)    0.0 (0.0)    3.03   
Leiber & Axelrod 1981          70.9 (76.7)    7.4 (7.0)    3.6 (4.0)    4.54   

                            G_Laland  diff_G  
                                              
Ramaley 1913                   13.19  -18.53  
Chamberlain 1928                5.09 -233.55  
Rife 1940                       4.37   -0.23  
Merrell 1957                    3.33  -24.93  
Annett 1973                     0.90  -13.67  
Ferronato et al. 1947           0.54   -0.23  
Mascie-Taylor (unpub)           0.11   -2.89  
Chaurasia & Goswani (unpub)     2.27  -18.02  
Annett 1978                     3.26   -5.24  
Carter-Saltzmann 1980           0.06   -2.48  
Coren & Porac 1980              2.25  -30.75  
McGee & Cozad 1980              3.80 -152.06  
McManus 1985 ICM1               0.01   -1.97  
McManus 1985 ICM2prop           2.61   -0.78  
McManus 1985 ICM2mat            0.88   -0.47  
McManus 1985 ICM2pat            0.01   -3.02  
Leiber & Axelrod 1981           1.75   -2.79

In [188]:
G_all

556.03

## Step 2: G-test with adjustments for MLE using the matrix T

### 3D

In [206]:
""" for estimated of  ρ=0.267, α=0.148, β=0.012 """

G_all, p_all, Delta_overall, G_each, p_each, Delta_each = Goodness_of_fit(thetas[0,:], D, adjust= True)

print('Delta overall=', Delta_overall)
print('Delta each dataset=', Delta_each)
print("Biggest diff =", abs(Delta_each).max())
print("Smallest diff =", abs(Delta_each).min())


res_step2_3D = results_table(thetas[0,:], D, adjust= True, delta= False)
res_step2_3D['diff_G'] = Delta_each
res_step2_3D

# res_step2_3D.to_excel('/Users/ronykarstadt/Desktop/Laland et at/results of G test feb 2024/results G-test step 2_3D.xlsx')

G-test for ρ=0.267, α=0.148, β=0.012
df overall =  31  df each =  1.82 

------------------------------
Overall:

G= 43.6,	p_value= 0.067
------------------------------
Each row in dataset:

G= 12.7*,	p_value= 0.001
G= 4.65,	p_value= 0.083
G= 3.91,	p_value= 0.121
G= 3.53,	p_value= 0.148
G= 1.08,	p_value= 0.536
G= 0.56,	p_value= 0.716
G= 0.11,	p_value= 0.927
G= 2.12,	p_value= 0.308
G= 3.43,	p_value= 0.156
G= 0.07,	p_value= 0.953
G= 2.21,	p_value= 0.295
G= 3.81,	p_value= 0.128
G= 0.0,	p_value= 1.000
G= 2.7,	p_value= 0.228
G= 0.88,	p_value= 0.598
G= 0.01,	p_value= 0.989
G= 1.8,	p_value= 0.365
------------------------------
Delta overall= 0.87
Delta each dataset= [ 0.51  0.44  0.46 -0.2  -0.18 -0.02  0.    0.15 -0.17 -0.01  0.04 -0.01
  0.01 -0.09  0.    0.   -0.05]
Biggest diff = 0.51
Smallest diff = 0.0


RxR                       RxL  \
                                         R            L            R   
Ramaley 1913                   841 (826.7)  115 (129.3)  113 (123.4)   
Chamberlain 1928             6917 (6906.9)  308 (318.1)  411 (419.7)   
Rife 1940                    1842 (1838.3)  151 (154.7)  140 (143.8)   
Merrell 1957                   140 (137.8)    34 (36.2)    33 (37.3)   
Annett 1973                  6206 (6214.3)  669 (660.7)  471 (469.9)   
Ferronato et al. 1947          154 (152.5)    11 (12.5)    31 (32.5)   
Mascie-Taylor (unpub)          232 (232.7)    17 (16.3)    41 (40.3)   
Chaurasia & Goswani (unpub)  1060 (1067.6)  144 (136.4)  122 (127.0)   
Annett 1978                  1656 (1652.5)  130 (133.5)  170 (174.7)   
Carter-Saltzmann 1980          303 (301.8)    37 (38.2)    45 (45.5)   
Coren & Porac 1980             315 (320.5)    68 (62.5)    57 (52.2)   
McGee & Cozad 1980             848 (828.0)  211 (231.0)  325 (335.5)   
McManus 1985 ICM1                58 (58.0)      9 (9.0)    14 (14.0)   
McManus 1985 ICM2prop          134 (131.7)    15 (17.3)    17 (19.6)   
McManus 1985 ICM2mat             74 (73.1)      4 (4.9)      6 (6.9)   
McManus 1985 ICM2pat             86 (85.9)      4 (4.1)      8 (8.1)   
Leiber & Axelrod 1981        1729 (1737.5)  173 (164.5)  281 (273.4)   

                                                LxL                 G diff_G  
                                       L          R          L                
Ramaley 1913                   54 (43.6)    1 (4.7)    7 (3.3)  12.68   0.51  
Chamberlain 1928               53 (44.3)  18 (21.1)    7 (3.9)   4.65   0.44  
Rife 1940                      34 (30.2)    5 (7.9)    6 (3.1)   3.91   0.46  
Merrell 1957                   20 (15.7)    8 (6.0)    2 (4.0)   3.53  -0.20  
Annett 1973                  125 (126.1)    5 (3.9)    1 (2.1)   1.08  -0.18  
Ferronato et al. 1947            9 (7.5)    0 (0.0)    0 (0.0)   0.56  -0.02  
Mascie-Taylor (unpub)            7 (7.7)    3 (2.9)    1 (1.1)   0.11   0.00  
Chaurasia & Goswani (unpub)    46 (41.0)    3 (4.2)    4 (2.8)   2.12   0.15  
Annett 1978                    40 (35.3)    4 (2.9)    0 (1.1)   3.43  -0.17  
Carter-Saltzmann 1980          15 (14.5)    0 (0.0)    0 (0.0)   0.07  -0.01  
Coren & Porac 1980             16 (20.8)    0 (0.0)    0 (0.0)   2.21   0.04  
McGee & Cozad 1980           150 (139.5)  30 (32.3)  22 (19.7)   3.81  -0.01  
McManus 1985 ICM1                5 (5.0)    0 (0.0)    0 (0.0)   0.00   0.01  
McManus 1985 ICM2prop            9 (6.4)    1 (0.6)    0 (0.4)   2.70  -0.09  
McManus 1985 ICM2mat             2 (1.1)    0 (0.0)    0 (0.0)   0.88   0.00  
McManus 1985 ICM2pat             1 (0.9)    0 (0.0)    0 (0.0)   0.01   0.00  
Leiber & Axelrod 1981          63 (70.6)    8 (7.2)    3 (3.8)   1.80  -0.05

### 2D

In [207]:
""" for estimated of  ρ=0.277, α=0.128, β=0 """

G_all, p_all, Delta_overall, G_each, p_each, Delta_each = Goodness_of_fit(thetas[1,:], D, adjust= True)

print('Delta overall=', Delta_overall)
print('Delta each dataset=', Delta_each)
print("Biggest diff =", abs(Delta_each).max())
print("Smallest diff =", abs(Delta_each).min())


res_step2_2D = results_table(thetas[1,:], D, adjust= True, delta= False)
res_step2_2D['diff_G'] = Delta_each
res_step2_2D

# res_step2_2D.to_excel('/Users/ronykarstadt/Desktop/Laland et at/results of G test feb 2024/results G-test step 2_2D.xlsx')

G-test for ρ=0.277, α=0.138, β=0.000
df overall =  32  df each =  1.88 

------------------------------
Overall:

G= 44.4,	p_value= 0.071
------------------------------
Each row in dataset:

G= 13.2*,	p_value= 0.001
G= 5.1,	p_value= 0.070
G= 4.37,	p_value= 0.102
G= 3.33,	p_value= 0.172
G= 0.91,	p_value= 0.606
G= 0.54,	p_value= 0.738
G= 0.11,	p_value= 0.934
G= 2.27,	p_value= 0.298
G= 3.26,	p_value= 0.179
G= 0.06,	p_value= 0.961
G= 2.25,	p_value= 0.300
G= 3.8,	p_value= 0.135
G= 0.0,	p_value= 1.000
G= 2.61,	p_value= 0.250
G= 0.88,	p_value= 0.614
G= 0.01,	p_value= 0.991
G= 1.75,	p_value= 0.389
------------------------------
Delta overall= 0.0
Delta each dataset= [ 0.   -0.01  0.    0.   -0.01  0.    0.    0.    0.    0.    0.    0.
  0.01  0.    0.    0.    0.  ]
Biggest diff = 0.01
Smallest diff = 0.0


RxR                       RxL  \
                                         R            L            R   
Ramaley 1913                   841 (826.7)  115 (129.3)  113 (123.3)   
Chamberlain 1928             6917 (6906.8)  308 (318.2)  411 (419.8)   
Rife 1940                    1842 (1838.3)  151 (154.7)  140 (143.8)   
Merrell 1957                   140 (137.8)    34 (36.2)    33 (37.2)   
Annett 1973                  6206 (6214.0)  669 (661.0)  471 (469.9)   
Ferronato et al. 1947          154 (152.6)    11 (12.4)    31 (32.5)   
Mascie-Taylor (unpub)          232 (232.7)    17 (16.3)    41 (40.3)   
Chaurasia & Goswani (unpub)  1060 (1067.7)  144 (136.3)  122 (126.8)   
Annett 1978                  1656 (1652.4)  130 (133.6)  170 (174.7)   
Carter-Saltzmann 1980          303 (301.8)    37 (38.2)    45 (45.5)   
Coren & Porac 1980             315 (320.5)    68 (62.5)    57 (52.1)   
McGee & Cozad 1980             848 (828.1)  211 (230.9)  325 (334.9)   
McManus 1985 ICM1                58 (58.0)      9 (9.0)    14 (14.0)   
McManus 1985 ICM2prop          134 (131.7)    15 (17.3)    17 (19.6)   
McManus 1985 ICM2mat             74 (73.1)      4 (4.9)      6 (6.9)   
McManus 1985 ICM2pat             86 (85.9)      4 (4.1)      8 (8.1)   
Leiber & Axelrod 1981        1729 (1737.5)  173 (164.5)  281 (273.1)   

                                                LxL                 G diff_G  
                                       L          R          L                
Ramaley 1913                   54 (43.7)    1 (4.9)    7 (3.1)  13.19   0.00  
Chamberlain 1928               53 (44.2)  18 (21.3)    7 (3.7)   5.10  -0.01  
Rife 1940                      34 (30.2)    5 (8.0)    6 (3.0)   4.37   0.00  
Merrell 1957                   20 (15.8)    8 (6.1)    2 (3.9)   3.33   0.00  
Annett 1973                  125 (126.1)    5 (4.0)    1 (2.0)   0.91  -0.01  
Ferronato et al. 1947            9 (7.5)    0 (0.0)    0 (0.0)   0.54   0.00  
Mascie-Taylor (unpub)            7 (7.7)    3 (3.0)    1 (1.0)   0.11   0.00  
Chaurasia & Goswani (unpub)    46 (41.2)    3 (4.4)    4 (2.6)   2.27   0.00  
Annett 1978                    40 (35.3)    4 (3.0)    0 (1.0)   3.26   0.00  
Carter-Saltzmann 1980          15 (14.5)    0 (0.0)    0 (0.0)   0.06   0.00  
Coren & Porac 1980             16 (20.9)    0 (0.0)    0 (0.0)   2.25   0.00  
McGee & Cozad 1980           150 (140.1)  30 (32.7)  22 (19.3)   3.80   0.00  
McManus 1985 ICM1                5 (5.0)    0 (0.0)    0 (0.0)   0.00   0.01  
McManus 1985 ICM2prop            9 (6.4)    1 (0.6)    0 (0.4)   2.61   0.00  
McManus 1985 ICM2mat             2 (1.1)    0 (0.0)    0 (0.0)   0.88   0.00  
McManus 1985 ICM2pat             1 (0.9)    0 (0.0)    0 (0.0)   0.01   0.00  
Leiber & Axelrod 1981          63 (70.9)    8 (7.4)    3 (3.6)   1.75   0.00

In [162]:
G_1995_each.sum()

44.42999999999999

## Step 3: G-test with adjustments for MLE using the matrix M

### 3D

In [199]:
""" for estimated of  ρ=0.207, α=0.203, β=0.042 """

G_all, p_all, Delta_overall, G_each, p_each, Delta_each = Goodness_of_fit(thetas[4,:], D, adjust= True)

print('Delta overall=', Delta_overall)
print('Delta each dataset=', Delta_each)
print("Biggest diff =", abs(Delta_each).max())
print("Smallest diff =", abs(Delta_each).min())

res_step3_3D = results_table(thetas[4,:], D, adjust= True, delta= False)
res_step3_3D['diff_G'] = Delta_each
res_step3_3D

# res_step3_3D.to_excel('/Users/ronykarstadt/Desktop/Laland et at/results of G test feb 2024/results G-test step 3_3D.xlsx')

G-test for ρ=0.207, α=0.203, β=0.042
df overall =  31  df each =  1.82 

------------------------------
Overall:

G= 36.3,	p_value= 0.234
------------------------------
Each row in dataset:

G= 7.1*,	p_value= 0.024
G= 3.31,	p_value= 0.166
G= 2.77,	p_value= 0.220
G= 4.63,	p_value= 0.084
G= 1.8,	p_value= 0.366
G= 0.48,	p_value= 0.745
G= 0.2,	p_value= 0.878
G= 2.11,	p_value= 0.311
G= 3.59,	p_value= 0.143
G= 0.04,	p_value= 0.973
G= 3.77,	p_value= 0.131
G= 0.63,	p_value= 0.686
G= 0.06,	p_value= 0.960
G= 2.33,	p_value= 0.276
G= 0.83,	p_value= 0.616
G= 0.01,	p_value= 0.993
G= 2.68,	p_value= 0.230
------------------------------
Delta overall= 8.1
Delta each dataset= [ 6.09  1.78  1.6  -1.3  -0.9   0.06 -0.09  0.16 -0.33  0.02 -1.52  3.17
 -0.05  0.28  0.05  0.   -0.93]
Biggest diff = 6.09
Smallest diff = 0.0


In [202]:
thetas[5,:]

array([0.239, 0.172, 0.   ])

### 2D

In [201]:
""" for estimated of  ρ=0.207, α=0.203, β=0.042 """

G_all, p_all, Delta_overall, G_each, p_each, Delta_each = Goodness_of_fit(thetas[5,:], D, adjust= True)

print('Delta overall=', Delta_overall)
print('Delta each dataset=', Delta_each)
print("Biggest diff =", abs(Delta_each).max())
print("Smallest diff =", abs(Delta_each).min())

res_step3_2D = results_table(thetas[5,:], D, adjust= True, delta= False)
res_step3_2D['diff_G'] = Delta_each
res_step3_2D

# res_step3_2D.to_excel('/Users/ronykarstadt/Desktop/Laland et at/results of G test feb 2024/results G-test step 3_2D.xlsx')

G-test for ρ=0.239, α=0.172, β=0.000
df overall =  32  df each =  1.88 

------------------------------
Overall:

G= 37.5,	p_value= 0.231
------------------------------
Each row in dataset:

G= 8.29*,	p_value= 0.014
G= 4.22,	p_value= 0.109
G= 3.79,	p_value= 0.137
G= 3.55,	p_value= 0.154
G= 1.35,	p_value= 0.481
G= 0.4,	p_value= 0.796
G= 0.19,	p_value= 0.891
G= 2.65,	p_value= 0.244
G= 3.04,	p_value= 0.201
G= 0.07,	p_value= 0.957
G= 4.14,	p_value= 0.114
G= 0.24,	p_value= 0.869
G= 0.09,	p_value= 0.943
G= 1.98,	p_value= 0.346
G= 0.81,	p_value= 0.636
G= 0.01,	p_value= 0.995
G= 2.69,	p_value= 0.239
------------------------------
Delta overall= 6.93
Delta each dataset= [ 4.9   0.87  0.58 -0.22 -0.45  0.14 -0.08 -0.38  0.22 -0.01 -1.89  3.56
 -0.08  0.63  0.07  0.   -0.94]
Biggest diff = 4.9
Smallest diff = 0.0
